### 1. Buscando o PUUID
<hr>
<p style="text-align: justify;"> 
Neste código, eu utilizei a <strong>Riot Games API</strong> para buscar meu <strong>'puuid'</strong>, usando o nome de invocador (<em>game_name</em>) e a (<em>tag_line</em>) como base. O PUUID (Player Universally Unique Identifier) é um identificador único global do jogador dentro da Riot Games. Ele é usado para identificar contas de forma consistente em todas as regiões e jogos da Riot. Eu precisei do PUUID para utilizá-lo como chave principal na busca de dados mais avançados, como o histórico de partidas e estatísticas detalhadas, essenciais para análises futuras. </p>

In [3]:
import requests

# Minha chave para acessar a API
api_key = "RGAPI-04ccf2bb-1c8c-46c1-8afc-c3648f605961"
# Essa chave é expirada no mesmo dia

game_name = "Suerdo" 
tag_line = "Br1"  

url = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}"

# Cabeçalhos da requisição
headers = {
    "X-Riot-Token": api_key  
}

response = requests.get(url, headers=headers)

# Verificar a resposta
if response.status_code == 200:
    data = response.json()  # Dados retornados pela API
    print("Informações do jogador:")
    print(data)
else:
    print(f"Erro {response.status_code}: {response.text}")

Informações do jogador:
{'puuid': 'n6tXroIhiXe8wLWPBM1j6BV6vq9VNztTEGZLMCFL5WINy7Dr6rQOD5NaVQYlsNrkKwYCSDBf9i5HCA', 'gameName': 'Suerdo', 'tagLine': 'BR1'}


### 2. Script para coleta de dados da Riot Games API
<hr>
<p style="text-align: justify;"> 
Neste código, utilizei novamente a Riot Games API para coletar o histórico de partidas do meu PUUID. Primeiro, busquei os IDs das partidas em lotes de até 100, respeitando os limites da API, até alcançar 1000 partidas ou esgotar as disponíveis. Em seguida, com os IDs, coletei os detalhes completos de cada partida, como estatísticas e resultados. Por fim, salvei todas as informações em um arquivo JSON (matches_data.json) para análises futuras.
</p>

In [9]:
import requests
import json
import time

# Minha chave da API
API_KEY = "RGAPI-04ccf2bb-1c8c-46c1-8afc-c3648f605961"

# Meu PUUID
PUUID = "n6tXroIhiXe8wLWPBM1j6BV6vq9VNztTEGZLMCFL5WINy7Dr6rQOD5NaVQYlsNrkKwYCSDBf9i5HCA"

# URL base da Riot API para dados de partidas
BASE_URL = "https://americas.api.riotgames.com"  # Servidor de dados de partidas

# Função para buscar IDs das partidas recentes com paginação
def get_match_ids(puuid, start=0, count=100):
    url = f"{BASE_URL}/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}"
    headers = {"X-Riot-Token": API_KEY}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro ao buscar IDs de partidas: {response.status_code} - {response.text}")
        return None

# Função para buscar detalhes de uma partida
def get_match_details(match_id):
    url = f"{BASE_URL}/lol/match/v5/matches/{match_id}"
    headers = {"X-Riot-Token": API_KEY}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro ao buscar detalhes da partida {match_id}: {response.status_code} - {response.text}")
        return None

# Main: Coletar dados de até 1000 partidas
def main():
    all_match_ids = []
    start = 0  # Início da busca
    batch_size = 100  # Número de partidas por requisição
    total_matches = 1000  # Número total de partidas que queremos buscar

    print("Coletando IDs de partidas...")

    # Buscar partidas em lotes até atingir o limite
    while len(all_match_ids) < total_matches:
        match_ids = get_match_ids(PUUID, start=start, count=batch_size)
        if not match_ids:
            break
        all_match_ids.extend(match_ids)
        start += batch_size

        print(f"Coletados {len(all_match_ids)} IDs de partidas até agora...")

        # Adicione um delay para respeitar os limites de requisoção da API da Riot
        time.sleep(1.2)  # Pequeno delay entre requisições

        # Parar se não houver mais partidas disponíveis
        if len(match_ids) < batch_size:
            break

    print(f"Total de IDs de partidas coletados: {len(all_match_ids)}")

    # Limitar ao máximo desejado
    all_match_ids = all_match_ids[:total_matches]

    # Buscar detalhes de cada partida
    matches_data = []
    for idx, match_id in enumerate(all_match_ids):
        print(f"Coletando detalhes da partida {idx + 1}/{len(all_match_ids)}: {match_id}")
        match_details = get_match_details(match_id)
        if match_details:
            matches_data.append(match_details)

        # Adicione um delay para evitar ultrapassar o limite de requisições
        time.sleep(1.2)

    # Salvar os dados em um arquivo JSON 
    with open("matches_data.json", "w") as f:
        json.dump(matches_data, f, indent=4)

    print("Dados das partidas salvos em 'matches_data.json'.")

if __name__ == "__main__":
    main()

Coletando IDs de partidas...
Coletados 100 IDs de partidas até agora...
Coletados 200 IDs de partidas até agora...
Coletados 300 IDs de partidas até agora...
Coletados 400 IDs de partidas até agora...
Coletados 500 IDs de partidas até agora...
Coletados 600 IDs de partidas até agora...
Coletados 700 IDs de partidas até agora...
Coletados 800 IDs de partidas até agora...
Coletados 900 IDs de partidas até agora...
Coletados 990 IDs de partidas até agora...
Total de IDs de partidas coletados: 990
Coletando detalhes da partida 1/990: BR1_3053562713
Coletando detalhes da partida 2/990: BR1_3053554735
Coletando detalhes da partida 3/990: BR1_3053543004
Coletando detalhes da partida 4/990: BR1_3053538520
Coletando detalhes da partida 5/990: BR1_3053539269
Coletando detalhes da partida 6/990: BR1_3053339286
Coletando detalhes da partida 7/990: BR1_3053319429
Coletando detalhes da partida 8/990: BR1_3053257235
Coletando detalhes da partida 9/990: BR1_3053138594
Coletando detalhes da partida 10/